# Data Acquisition and Cleaning the Data

I have requested the data from the CDC Database using sodapy which is the python client of Socrata Open Data API.

## Step 1: Requesting the Data

 1. The dataset were requested using their unique ID from the CDC Database. 
 2. SQL was used to filter specific data of interest for parsing. 
 3. The data was returned in a json format with a limit of 20,000 data entries.
 4. The json formatted dataset were converted into individual pd data frames.  

## Step 2. Cleaning the Parsed Data and Outputing them into CSV Files
 1. The variables within the data frames were changed for easier readable later on. 
 2. A couple of the columns from the original datasets were dropped since they are irrevelant to this report analysis. 
 3. Output csv files for each dataset:
     <br>a. ***Death_Counts_Race_Group.csv***
     <br>b. ***Vac_Race.csv***
     <br>c. ***Covid_Death_State_Counties_level.csv***

In [264]:
import requests
import requests_cache
import pandas as pd
import numpy as np
import csv
from sodapy import Socrata
import urllib
import json

# Ignore the warning banner after the cells
import warnings
warnings.filterwarnings('ignore')
requests_cache.install_cache('covid_cache')

In [254]:
# Get rid of the pink warning boxes

#Hide the Pink Warning sign boxes

from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

## Step 1: Data Acquisition

In [255]:
def cdc_data(ID, param):
    # parsing data from the cdc.gov website
    url = "data.cdc.gov"
    client = Socrata(url, None)
    # returning the dataset in json format and limiting data to only 20,000 rows
    results = client.get(ID, limit=20000)
    # put json data into panda data frame for easier usage
    df = pd.DataFrame.from_records(results)
    # return the df based on the parameters that are passed through
    if param == "":
        return df  
    else:        
        df2 = df.query(param)
        return df2
    
    

## Step 2: Cleaning the Parsed Data and Output Data into CSV

### Covid-19 Death Counts Based on Racial Groups

In [256]:
DC = cdc_data("pj7m-y5uh", 'group =="By Month" and indicator =="Count of COVID-19 deaths"')

DC.columns = ['data_as_of', 
              'start_week', 
              'end_week', 
              'year',
              'group', 
              'state',
              'indicator', 
              "white", 
         "black_american",
        "american_indian_alaska native",
        "asian_pacific_islander",
        "other_pacific_islander",
        "mixed_race",
        "hispanic_latino", 'month','footnote']
DC_df = DC.drop(['data_as_of','footnote'], axis=1)
DC_df.to_csv('Death_Counts_Race_Group.csv')
DC_df.head()

,start_week,end_week,year,group,state,indicator,white,black_american,american_indian_alaska native,asian_pacific_islander,other_pacific_islander,mixed_race,hispanic_latino,month
12,2020-01-01T00:00:00.000,2020-01-31T00:00:00.000,2020,By Month,United States,Count of COVID-19 deaths,3,3,0,0,0,0,1,1
16,2020-02-01T00:00:00.000,2020-02-29T00:00:00.000,2020,By Month,United States,Count of COVID-19 deaths,13,1,0,1,0,0,3,2
20,2020-03-01T00:00:00.000,2020-03-31T00:00:00.000,2020,By Month,United States,Count of COVID-19 deaths,3241,2173,30,444,6,19,1124,3
24,2020-04-01T00:00:00.000,2020-04-30T00:00:00.000,2020,By Month,United States,Count of COVID-19 deaths,33154,16371,299,3463,41,159,11066,4
28,2020-05-01T00:00:00.000,2020-05-31T00:00:00.000,2020,By Month,United States,Count of COVID-19 deaths,21769,7836,439,1642,38,89,6225,5


### Vaccination Counts Based on Racial Groups


In [257]:
# This function help get only the subgroups we are interested in 
def get_demo(df,col,demograph):
    demo_data = df.loc[df[col].isin(demograph)]
    return demo_data

In [258]:
vac= cdc_data("km4m-vcsb", "")

Group=('Race_eth_NHBlack', 
       'Race_eth_unknown', 'US',
       'Race_eth_NHNHOPI', 'Race_eth_Hispanic', 'Race_eth_NHWhite',
       'Race_eth_NHAIAN', 'Race_eth_NHAsian', 'Race_eth_NHMult_Oth',
        'Race_eth_known' )

vac_df = get_demo(vac, "demographic_category", Group)

vac_df['demographic_category'] = vac_df['demographic_category'].replace({'Race_eth_Hispanic':'hispanic/latino',
                                                 'Race_eth_NHWhite':'white',
                                                'Race_eth_NHAIAN':'american_indian/alaska_native',
                                                'Race_eth_NHAsian':'asian',
                                                'Race_eth_NHBlack':'black',
                                                'Race_eth_NHNHOPI':'native_hawaiian/other_pacific_islander',
                                                'Race_eth_NHMult_Oth':'other_race_or_multi-racial',
                                                'Race_eth_unknown':'unknown',
                                                'Race_eth_known':'known'})


vac_df.to_csv('Vac_Race.csv')
vac_df.head()

,date,demographic_category,administered_dose1,administered_dose1_pct_known,administered_dose1_pct_us,series_complete_yes,administered_dose1_pct,series_complete_pop_pct,series_complete_pop_pct_known,series_complete_pop_pct_us
0,2021-06-10T00:00:00.000,black,8883539,9,0,7537560,23.6,20,8.7,0
4,2021-06-10T00:00:00.000,unknown,60948707,62,0,43828721,0,0,50.4,0
7,2021-06-10T00:00:00.000,US,159198695,0,0,130796141,0,0,0,0
8,2021-06-10T00:00:00.000,native_hawaiian/other_pacific_islander,295066,0.3,0,241498,33.1,27.1,0.3,0
13,2021-06-10T00:00:00.000,hispanic/latino,14383057,14.6,0,11664320,27.5,22.3,13.4,0


### Total Covid-19 Death Counts by Counties as of June-8-2021

In [268]:
url_ = "https://ephtracking.cdc.gov/apigateway/api/v1/getCoreHolder/915/2/all/all/20210608/0/0"
count_covid = requests.get(url_)
jss = count_covid.json()
CCS_df = pd.DataFrame(jss["dayTableResult"])
CCS = CCS_df[['id','dataValue',
              'geo',
              'parentGeo',
              'geoId',
              'parentGeoId',
              'geoAbbreviation',
              'parentGeoAbbreviation']]
CCS.to_csv('Covid_Death_State_Counties_level.csv')
CCS.head()

,id,dataValue,geo,parentGeo,geoId,parentGeoId,geoAbbreviation,parentGeoAbbreviation
0,3164923,113,Autauga,Alabama,01001,01,01001,AL
1,3165843,312,Baldwin,Alabama,01003,01,01003,AL
2,3166752,59,Barbour,Alabama,01005,01,01005,AL
3,3167652,64,Bibb,Alabama,01007,01,01007,AL
4,3168543,139,Blount,Alabama,01009,01,01009,AL
